# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [1]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [2]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def get_llm_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """Get a response from the LLM based on the prompt"""
    # TODO: Implement the get_llm_response function
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_length=256)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# TODO: Test your get_llm_response function with different prompts

if __name__ == "__main__":
    print("Zero-shot:\n", get_llm_response(zero_shot_prompt))
    print("\nOne-shot:\n", get_llm_response(one_shot_prompt))
    print("\nFew-shot:\n", get_llm_response(few_shot_prompt))

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-21 02:50:01.857503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747795802.199418   48691 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747795802.307482   48691 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747795802.977456   48691 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000

Zero-shot:
 ice cream

One-shot:
 Gout is a bacterial infection that causes inflammation of the urethra and spleen.

Few-shot:
 Gout is a bacterial infection that causes inflammation and irritability in the gastrointestinal tract.


## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [7]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# TODO: Compare the different prompting strategies on these questions
# For each question:
# - Create prompts using each strategy
# - Get responses from the LLM
# - Store the results

def try_zero_shot(q):
    return f"Question: {q}\nAnswer:"

def try_one_shot(q):
    return f"""Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {q}
Answer:"""

def try_few_shot(q):
    return f"""Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {q}
Answer:"""

# Store results
results = []

for q in questions:
    z_prompt = try_zero_shot(q)
    o_prompt = try_one_shot(q)
    f_prompt = try_few_shot(q)

    z_resp = get_llm_response(z_prompt)
    o_resp = get_llm_response(o_prompt)
    f_resp = get_llm_response(f_prompt)

    results.append({
        "question": q,
        "zero_shot": z_resp,
        "one_shot": o_resp,
        "few_shot": f_resp
    })

# Display results
for r in results:
    print(f"\n{r['question']}")
    print("Zero-shot:", r["zero_shot"])
    print("One-shot:", r["one_shot"])
    print("Few-shot:", r["few_shot"])


What foods should be avoided by patients with gout?
Zero-shot: ice cream
One-shot: Gout is a bacterial infection that causes inflammation of the urethra and spleen.
Few-shot: Gout is a bacterial infection that causes inflammation and irritability in the gastrointestinal tract.

What medications are commonly prescribed for gout?
Zero-shot: acetaminophen
One-shot: Gout is a bacterial infection that causes inflammation in the gout joint.
Few-shot: Gout is a bacterial infection that causes inflammation in the urethra and urethra.

How can gout flares be prevented?
Zero-shot: if you have a gout flare, you should take a shower and eat a gout cake.
One-shot: Gout flares can be prevented by avoiding foods that are high in salt and sugar.
Few-shot: Gout flares can be prevented by avoiding foods that are high in uric acid.

Is gout related to diet?
Zero-shot: no
One-shot: Gout is a bacterial infection that can be caused by a variety of causes, including a lack of food or water.
Few-shot: Gout i

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [12]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    # Example implementation:
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
# Determine which strategy performs best overall

zero_shot_total = 0
one_shot_total = 0
few_shot_total = 0
num_questions = len(results)

for r in results:
    question = r["question"]
    keywords = expected_keywords.get(question, [])

    z_score = score_response(r["zero_shot"], keywords)
    o_score = score_response(r["one_shot"], keywords)
    f_score = score_response(r["few_shot"], keywords)

    r["zero_shot_score"] = z_score
    r["one_shot_score"] = o_score
    r["few_shot_score"] = f_score

    zero_shot_total += z_score
    one_shot_total += o_score
    few_shot_total += f_score

print("\nAverage Scores:")
print(f"Zero-shot average score: {zero_shot_total / num_questions:.2f}")
print(f"One-shot average score: {one_shot_total / num_questions:.2f}")
print(f"Few-shot average score: {few_shot_total / num_questions:.2f}")




Average Scores:
Zero-shot average score: 0.00
One-shot average score: 0.03
Few-shot average score: 0.03


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [ ]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""


from pathlib import Path

def save_prompting_results(results, output_file="results/part_3/prompt_comparison.txt"):
    """Save prompting experiment results including responses and scores"""
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_file, "w") as f:
        f.write("# Prompt Engineering Results\n\n")

        for r in results:
            f.write(f"## Question: {r['question']}\n\n")
            f.write("### Zero-shot response:\n")
            f.write(f"{r['zero_shot']}\n\n")
            f.write("### One-shot response:\n")
            f.write(f"{r['one_shot']}\n\n")
            f.write("### Few-shot response:\n")
            f.write(f"{r['few_shot']}\n\n")
            f.write("-" * 50 + "\n\n")

        f.write("## Scores\n\n")
        f.write("```\n")
        f.write("question,zero_shot,one_shot,few_shot\n")

        total_zero = total_one = total_few = 0
        for r in results:
            q_id = r['question'].lower().replace('?', '').replace(' ', '_')
            zs, os, fs = r['zero_shot_score'], r['one_shot_score'], r['few_shot_score']
            total_zero += zs
            total_one += os
            total_few += fs
            f.write(f"{q_id},{zs:.2f},{os:.2f},{fs:.2f}\n")

        n = len(results)
        avg_zero = total_zero / n
        avg_one = total_one / n
        avg_few = total_few / n
        f.write(f"\naverage,{avg_zero:.2f},{avg_one:.2f},{avg_few:.2f}\n")

        best_method = max(
            [("zero_shot", avg_zero), ("one_shot", avg_one), ("few_shot", avg_few)],
            key=lambda x: x[1]
        )[0]
        f.write(f"best_method,{best_method}\n")
        f.write("```\n")


save_prompting_results(results)


## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method